# Loop 1 Analysis: Fix Overlapping Trees

The submission failed due to 'Overlapping trees in group 002'. We need to:
1. Verify which groups have overlaps
2. Fix them by replacing with valid configurations
3. Create a valid submission

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 25
scale_factor = Decimal("1e18")

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""
    
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        
        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h
        
        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )

def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False
    
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def load_trees_for_n(n, df):
    """Load all trees for a given N from the submission DataFrame."""
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"]).lstrip('s')
        y = str(row["y"]).lstrip('s')
        deg = str(row["deg"]).lstrip('s')
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print("Functions defined successfully")

In [ ]:
# Load the submitted file and check for overlaps
df = pd.read_csv('/home/code/submission_candidates/candidate_000.csv')
print(f"Loaded {len(df)} rows")

# Find all invalid groups
invalid_groups = []
for n in range(1, 201):
    trees = load_trees_for_n(n, df)
    if trees and has_overlap(trees):
        invalid_groups.append(n)
        print(f"Group {n:03d} has overlapping trees!")

print(f"\nTotal invalid groups: {len(invalid_groups)}")
print(f"Invalid groups: {invalid_groups}")

In [ ]:
# Let's look at the specific group 002 that failed
group_002 = df[df['id'].str.startswith('002_')]
print("Group 002 data:")
print(group_002)

# Parse values
def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

for _, row in group_002.iterrows():
    x = parse_value(row['x'])
    y = parse_value(row['y'])
    deg = parse_value(row['deg'])
    print(f"  Tree {row['id']}: x={x:.6f}, y={y:.6f}, deg={deg:.6f}")

In [ ]:
# Check the original snapshot that was used
df_orig = pd.read_csv('/home/nonroot/snapshots/santa-2025/21328310479/submission/submission.csv')
print("Original snapshot group 002:")
group_002_orig = df_orig[df_orig['id'].str.startswith('002_')]
print(group_002_orig)

# Check if original has overlaps
trees_orig = load_trees_for_n(2, df_orig)
print(f"\nOriginal group 002 has overlap: {has_overlap(trees_orig)}")

In [ ]:
# Let's check the sample submission for a valid N=2 configuration
df_sample = pd.read_csv('/home/data/sample_submission.csv')
print("Sample submission group 002:")
group_002_sample = df_sample[df_sample['id'].str.startswith('002_')]
print(group_002_sample)

# Check if sample has overlaps
trees_sample = load_trees_for_n(2, df_sample)
print(f"\nSample group 002 has overlap: {has_overlap(trees_sample)}")

In [ ]:
# Let's find a snapshot with valid configurations for all groups
import os

snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
snapshots = sorted([d for d in os.listdir(snapshot_dir) if d.isdigit()])

print(f"Found {len(snapshots)} snapshots")

# Check the most recent snapshots for valid configurations
valid_snapshot = None
for snap in reversed(snapshots[-10:]):
    snap_path = os.path.join(snapshot_dir, snap, 'submission/submission.csv')
    if os.path.exists(snap_path):
        df_snap = pd.read_csv(snap_path)
        invalid = []
        for n in range(1, 201):
            trees = load_trees_for_n(n, df_snap)
            if trees and has_overlap(trees):
                invalid.append(n)
        if len(invalid) == 0:
            print(f"Snapshot {snap}: ALL VALID!")
            valid_snapshot = snap_path
            break
        else:
            print(f"Snapshot {snap}: {len(invalid)} invalid groups: {invalid[:5]}...")

In [ ]:
# Check more snapshots to find one with all valid groups
for snap in reversed(snapshots):
    snap_path = os.path.join(snapshot_dir, snap, 'submission/submission.csv')
    if os.path.exists(snap_path):
        df_snap = pd.read_csv(snap_path)
        invalid = []
        for n in range(1, 201):
            trees = load_trees_for_n(n, df_snap)
            if trees and has_overlap(trees):
                invalid.append(n)
        if len(invalid) == 0:
            print(f"Snapshot {snap}: ALL VALID!")
            valid_snapshot = snap_path
            break
    if valid_snapshot:
        break

if valid_snapshot:
    print(f"\nUsing valid snapshot: {valid_snapshot}")
else:
    print("No fully valid snapshot found!")

In [ ]:
# Calculate score for the valid snapshot
if valid_snapshot:
    df_valid = pd.read_csv(valid_snapshot)
    
    # Calculate score
    TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
    TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])
    
    def score_group_fast(xs, ys, degs):
        n = len(xs)
        all_x, all_y = [], []
        for i in range(n):
            rad = np.radians(degs[i])
            c, s = np.cos(rad), np.sin(rad)
            for j in range(len(TX)):
                x = TX[j] * c - TY[j] * s + xs[i]
                y = TX[j] * s + TY[j] * c + ys[i]
                all_x.append(x)
                all_y.append(y)
        side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
        return side * side / n
    
    total_score = 0
    for n in range(1, 201):
        group = df_valid[df_valid['id'].str.startswith(f'{n:03d}_')]
        xs = group['x'].apply(parse_value).values
        ys = group['y'].apply(parse_value).values
        degs = group['deg'].apply(parse_value).values
        total_score += score_group_fast(xs, ys, degs)
    
    print(f"Valid snapshot score: {total_score:.6f}")

In [ ]:
# Let's try a different approach - use the sample submission as baseline and check its score
df_sample = pd.read_csv('/home/data/sample_submission.csv')

# Check if sample has any overlaps
sample_invalid = []
for n in range(1, 201):
    trees = load_trees_for_n(n, df_sample)
    if trees and has_overlap(trees):
        sample_invalid.append(n)

print(f"Sample submission invalid groups: {len(sample_invalid)}")
if sample_invalid:
    print(f"Invalid: {sample_invalid[:10]}...")

# Calculate sample score
total_sample_score = 0
for n in range(1, 201):
    group = df_sample[df_sample['id'].str.startswith(f'{n:03d}_')]
    xs = group['x'].apply(parse_value).values
    ys = group['y'].apply(parse_value).values
    degs = group['deg'].apply(parse_value).values
    total_sample_score += score_group_fast(xs, ys, degs)

print(f"Sample submission score: {total_sample_score:.6f}")

In [ ]:
# Let's check the 21198893057 snapshot that was mentioned in state.json
snap_path = '/home/nonroot/snapshots/santa-2025/21198893057/submission/submission.csv'
if os.path.exists(snap_path):
    df_snap = pd.read_csv(snap_path)
    invalid = []
    for n in range(1, 201):
        trees = load_trees_for_n(n, df_snap)
        if trees and has_overlap(trees):
            invalid.append(n)
    print(f"Snapshot 21198893057: {len(invalid)} invalid groups")
    if invalid:
        print(f"Invalid: {invalid}")
    
    # Calculate score
    total_score = 0
    for n in range(1, 201):
        group = df_snap[df_snap['id'].str.startswith(f'{n:03d}_')]
        xs = group['x'].apply(parse_value).values
        ys = group['y'].apply(parse_value).values
        degs = group['deg'].apply(parse_value).values
        total_score += score_group_fast(xs, ys, degs)
    print(f"Score: {total_score:.6f}")

In [ ]:
# Let's find the best valid snapshot by checking all of them
best_valid_score = float('inf')
best_valid_path = None

for snap in snapshots:
    snap_path = os.path.join(snapshot_dir, snap, 'submission/submission.csv')
    if not os.path.exists(snap_path):
        continue
    
    try:
        df_snap = pd.read_csv(snap_path)
        
        # Check for overlaps
        has_invalid = False
        for n in range(1, 201):
            trees = load_trees_for_n(n, df_snap)
            if trees and has_overlap(trees):
                has_invalid = True
                break
        
        if has_invalid:
            continue
        
        # Calculate score
        total_score = 0
        for n in range(1, 201):
            group = df_snap[df_snap['id'].str.startswith(f'{n:03d}_')]
            xs = group['x'].apply(parse_value).values
            ys = group['y'].apply(parse_value).values
            degs = group['deg'].apply(parse_value).values
            total_score += score_group_fast(xs, ys, degs)
        
        if total_score < best_valid_score:
            best_valid_score = total_score
            best_valid_path = snap_path
            print(f"New best valid: {snap} with score {total_score:.6f}")
    except Exception as e:
        continue

print(f"\nBest valid snapshot: {best_valid_path}")
print(f"Best valid score: {best_valid_score:.6f}")

In [ ]:
# Copy the best valid submission to the submission folder
import shutil

if best_valid_path:
    # Copy to submission folder
    shutil.copy(best_valid_path, '/home/submission/submission.csv')
    print(f"Copied {best_valid_path} to /home/submission/submission.csv")
    
    # Also save to experiments folder
    os.makedirs('/home/code/experiments/003_valid_baseline', exist_ok=True)
    shutil.copy(best_valid_path, '/home/code/experiments/003_valid_baseline/submission.csv')
    
    # Save metrics
    import json
    with open('/home/code/experiments/003_valid_baseline/metrics.json', 'w') as f:
        json.dump({'cv_score': best_valid_score}, f)
    
    print(f"\nExperiment 003_valid_baseline created with score {best_valid_score:.6f}")
else:
    print("No valid snapshot found!")

In [ ]:
# Verify the submission is valid
df_final = pd.read_csv('/home/submission/submission.csv')

# Check for overlaps
final_invalid = []
for n in range(1, 201):
    trees = load_trees_for_n(n, df_final)
    if trees and has_overlap(trees):
        final_invalid.append(n)

print(f"Final submission invalid groups: {len(final_invalid)}")
if final_invalid:
    print(f"Invalid: {final_invalid}")
else:
    print("All groups are valid!")

# Calculate final score
total_final_score = 0
for n in range(1, 201):
    group = df_final[df_final['id'].str.startswith(f'{n:03d}_')]
    xs = group['x'].apply(parse_value).values
    ys = group['y'].apply(parse_value).values
    degs = group['deg'].apply(parse_value).values
    total_final_score += score_group_fast(xs, ys, degs)

print(f"\nFinal submission score: {total_final_score:.6f}")
print(f"Target score: 68.894234")
print(f"Gap: {total_final_score - 68.894234:.6f} ({(total_final_score - 68.894234) / 68.894234 * 100:.2f}%)")

In [ ]:
# Save candidate for submission
import shutil
os.makedirs('/home/code/submission_candidates', exist_ok=True)
shutil.copy('/home/submission/submission.csv', '/home/code/submission_candidates/candidate_001.csv')
print("Saved candidate_001.csv")